In [2]:
from splinter import Browser
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
def scrape():
    
    mars_dict = {}
    
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    
    url1 = 'https://mars.nasa.gov/news/'
    browser.visit(url1)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    news_title = soup.find('div', class_='content_title').text
    news_p = soup.find('div', class_='article_teaser_body').text
    
    url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url2)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    browser.links.find_by_partial_text('FULL IMAGE').click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    img_url = soup.find_all('img', class_='fancybox-image')[0]['src']
    featured_image_url = f'https://www.jpl.nasa.gov{img_url}'
    
    url3 = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(url3)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    mars_weather = soup.find('article').text
    mars_weather = mars_weather.replace('Mars Weather@MarsWxReportÂ·15h', '')
    
    url4 = 'https://space-facts.com/mars/'
    mars_table = pd.read_html(url4)
    mars_df = mars_table[0]
    table_string = mars_df.to_html()
    
    url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url5)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    hemispheres = soup.find_all('div', class_='item')
    hemisphere_image_urls = []

    for hemisphere in hemispheres:
        image = hemisphere.find('a', class_='itemLink product-item')['href']
        title = hemisphere.find('h3').text
        full_url = 'https://astrogeology.usgs.gov' + image
        browser.visit(full_url)
        html2 = browser.html
        soup2 = BeautifulSoup(html2, 'html.parser')
        img_url = 'https://astrogeology.usgs.gov' + soup2.find("img", class_="wide-image")["src"]
        hemisphere_image_urls.append({'title': title, 'img_url': img_url})
    
    mars_dict['news_title'] = news_title
    mars_dict['news_p'] = news_p
    mars_dict['featured_image_url'] = featured_image_url
    mars_dict['mars_weather'] = mars_weather
    mars_dict['mars_df'] = mars_df
    mars_dict['table_string'] = table_string
    mars_dict['hemisphere_image_urls'] = hemisphere_image_urls 
    
    return mars_dict
